In [4]:
from scipy.stats import norm
import numpy as np
from statsmodels.tsa.stattools import acovf

def newey_west_errors(y):
    sample_var = np.var(y)
    autocov = acovf(y)
    weights  = np.array([1-j/T for j in range(1,T)])
    adjusted_var = np.sum(weights * autocov) + sample_var 
    return sample_var, adjusted_var

def simulate_ar1(rho, sigma, T, reps, alpha=0.05, random_seed=1234):
    """
    Simulates an AR(1) process y_t = rho*y_{t-1} + eps_t (eps ~ N(0,sigma^2)).
    Tests H0: E[y_t] = 0 against H1: E[y_t] != 0.
    Returns the empirical rejection frequency.
    """
    np.random.seed(random_seed)
    reject_count = 0
    critical_value_2_sides = norm.ppf(1 - alpha/2)
    
    for _ in range(reps):
        # Generate data
        e = np.random.normal(0, sigma, T)
        y = np.zeros(T)
        for t in range(1, T):
            y[t] = rho * y[t-1] + e[t]
                
        sample_mean = np.mean(y)
        sample_var = np.var(y)
        
        # Compute t-statistic
        t_stat = sample_mean / np.sqrt(sample_var/T)
        
        # Check if t-statistic is greater than critical value
        if np.abs(t_stat) > critical_value_2_sides:
            reject_count += 1
        
    return reject_count / reps


# Parameters
rho = 0.8      # True phi under H0
sigma = 0.1    # Std dev of eps
T = 50         # Sample size
reps = 10000   # Number of Monte Carlo replications

for seed in range(10):
    empirical_size = simulate_ar1(rho, sigma, T, reps, random_seed=37*seed**2)
    print(f"Empirical rejection frequency at nominal 5% = {empirical_size*100:.2f}%")

Empirical rejection frequency at nominal 5% = 54.61%
Empirical rejection frequency at nominal 5% = 53.45%
Empirical rejection frequency at nominal 5% = 53.43%
Empirical rejection frequency at nominal 5% = 54.28%
Empirical rejection frequency at nominal 5% = 53.57%
Empirical rejection frequency at nominal 5% = 54.07%
Empirical rejection frequency at nominal 5% = 54.41%
Empirical rejection frequency at nominal 5% = 54.59%
Empirical rejection frequency at nominal 5% = 54.24%
Empirical rejection frequency at nominal 5% = 53.35%


In [5]:
# Check for gaussian that rej. rate=0.5%:
rho=0
for seed in range(10):
    empirical_size = simulate_ar1(rho, sigma, T, reps, random_seed=37*seed**2)
    print(f"Empirical rejection frequency at nominal 5% = {empirical_size*100:.2f}%")

Empirical rejection frequency at nominal 5% = 5.68%
Empirical rejection frequency at nominal 5% = 5.95%
Empirical rejection frequency at nominal 5% = 5.55%
Empirical rejection frequency at nominal 5% = 5.61%
Empirical rejection frequency at nominal 5% = 5.41%
Empirical rejection frequency at nominal 5% = 5.73%
Empirical rejection frequency at nominal 5% = 5.40%
Empirical rejection frequency at nominal 5% = 5.66%
Empirical rejection frequency at nominal 5% = 5.33%
Empirical rejection frequency at nominal 5% = 5.55%


In [8]:
# Compute errors
T = 5000
for seed in range(10):
    np.random.seed(37*seed**2)
    e = np.random.normal(0, sigma, T)
    y = np.zeros(T)
    for t in range(1, T):
        y[t] = rho * y[t-1] + e[t]
    print("Sample Var" ,newey_west_errors(y)[0], "\tAdjusted Var", newey_west_errors(y)[1])

Sample Var 0.009702557567966022 	Adjusted Var 0.006825298838599346
Sample Var 0.009844004266610685 	Adjusted Var 0.00543274449653488
Sample Var 0.01010460637294522 	Adjusted Var 0.0054708619741310385
Sample Var 0.009610506748650596 	Adjusted Var 0.00519011651459303
Sample Var 0.01001888016939426 	Adjusted Var 0.010962976084914401
Sample Var 0.009950890897207744 	Adjusted Var 0.005697114404920476
Sample Var 0.010256293387400589 	Adjusted Var 0.005731254372109187
Sample Var 0.009926258815127285 	Adjusted Var 0.011263666456381386
Sample Var 0.009964704570126671 	Adjusted Var 0.010865573602025613
Sample Var 0.0102059417147313 	Adjusted Var 0.006316204983909915
